## Description
This notebook processes all labeled entries by passing them through a prompt to the large language model (LLM) Google Gemini. It handles the data in batches of 10 rows from the DataFrame and waits for the result after each batch.

#### Improvments to consider
- Create multiple API keys to allow more requests per minute and per day.
- The prompt is somewhat unstable, as multiple entries are processed in a single request. Processing one entry at a time with max_output_tokens = 1 would be much more stable.
- Define promps in other languages

The code was created with the assistance of ChatGPT-4.

In [44]:
import pandas as pd
from google import genai
from google.genai import types
import json
from tqdm import tqdm
from itertools import cycle
import time


inputdata_file = '../02_Prototype_LM_BERT/data/03_labelled_data.csv' #currently the same file as in 02_Prototype_LM_BERT
outputdata_file ='data/02_predicted_data.csv'

with open("data/apikeys.json") as f:
    config = json.load(f)
API_KEYS = config["GOOGLE_API_KEYS"]
API_KEYS_CYCLE = cycle(API_KEYS)

In [45]:
df = pd.read_csv(inputdata_file, dtype={'mobilitydata_labelled': 'string'}, low_memory=False)

# Drop rows where 'mobilitydata_labelled' is empty (NaN)
df = df.dropna(subset=['mobilitydata_labelled'])

# Convert 'mobilitydata_labelled' to boolean type
df['mobilitydata_labelled'] = df['mobilitydata_labelled'].map({'True': True, 'False': False})

# Print the number of rows remaining after filtering
print(f"Number of labelled rows after filtering: {len(df)}")

Number of labelled rows after filtering: 150


In [ ]:
# Configuration
requests_per_key = 15  # max requests per key per minute
key_count = len(API_KEYS)
current_key_index = 0
key_request_counter = 0
cycle_start_time = time.time()


def process_chunks(df, indices, chunk_size):
    global current_key_index, key_request_counter, cycle_start_time

    for i in tqdm(range(0, len(indices), chunk_size)):
        # Rotate key if limit reached
        if key_request_counter >= requests_per_key:
            current_key_index += 1
            key_request_counter = 0

            if current_key_index >= key_count:
                elapsed = time.time() - cycle_start_time
                if elapsed < 60:
                    wait_time = int(60 - elapsed)
                    print(f"Max requests per minute reached. Waiting {wait_time} seconds...")
                    time.sleep(wait_time + 1)
                current_key_index = 0
                cycle_start_time = time.time()

        CURRENT_API_KEY = API_KEYS[current_key_index]
        client = genai.Client(api_key=CURRENT_API_KEY)

        # Prepare data
        batch_indices = indices[i:i + chunk_size]
        chunk_df = df.loc[batch_indices][['dataset_title_DE', 'dataset_description_DE']]

        chunk_lines = chunk_df.apply(
            lambda row: f"Titel: {row['dataset_title_DE']}\nBeschreibung: {row['dataset_description_DE']}",
            axis=1
        ).tolist()

        # prompt = (
        #     "Handelt es sich bei folgendem Inhalt um Verkehrs- oder Mobilitätsdaten?\n\n" +
        #     "\n\n".join(chunk_lines) + "\n\n" +
        #     "Antworte nur mit T oder F für True und False und reihe alle Antworten direkt aneinander. "
        #     "Ohne Leerzeichen, Umbrüche, Texte, Sonderzeichen."
        # )

        # prompt = (
        #     "Handelt es sich bei folgendem Inhalt um Verkehrs- oder Mobilitätsdaten?\n\n" +
        #     "\n\n".join(chunk_lines) + "\n\n" +
        #     "Antworte nur mit T oder F für True und False und reihe alle Antworten direkt aneinander ohne Leerzeichen, Umbrüche, Texte oder Sonderzeichen zu verwenden. Beispielhaftes Antwortschema:\n\n" +
        #     "TFTFFTFTTT"
        # )

        # prompt = (
        #     "Untenstehend finden Sie jeweils einen Titel und eine Beschreibung eines Datensatzes. "
        #     "Bewerten Sie für jede Kombination, ob es sich um Verkehrs- oder Mobilitätsdaten handelt.\n\n"
        #     "Antworten Sie für jede Zeile **nur** mit:\n"
        #     "- **T** für True = es handelt sich um Mobilitätsdaten\n"
        #     "- **F** für False = es handelt sich **nicht** um Mobilitätsdaten\n\n"
        #     "Reihen Sie alle Antworten ohne Leerzeichen, Umbrüche oder andere Zeichen direkt hintereinander. "
        #     "Geben Sie ausschließlich eine Zeichenkette bestehend aus T und F zurück.\n\n"
        #     "Beispiel für 10 Einträge:\n"
        #     "TFTFTFTFFT\n\n"
        #     "Datensätze:\n\n" +
        #     "\n\n".join(chunk_lines)
        # )

        # prompt = (
        #     "Entscheide für jeden der folgenden Datensätze, ob es sich um Verkehrs- oder Mobilitätsdaten handelt.\n\n" +
        #     "\n\n".join(chunk_lines) + "\n\n" +
        #     "Antworte nur mit T für 'True' (Mobilitätsdaten) oder F für 'False' (keine Mobilitätsdaten).\n"
        #     "Gib ausschließlich eine Zeichenkette aus T und F zurück – **ohne Leerzeichen, Umbrüche oder andere Zeichen**.\n\n"
        #     "Beispielantwort für zehn Datensätze:\nTFTFFTFTTT"
        # )

        # prompt = "Handelt es sich bei folgendem Inhalt um Verkehrs- oder Mobilitätsdaten? Antworte nur mit T (True) oder F (False) Zeilenweise.\n\n" + "\n\n".join(chunk_lines)
        
        prompt = "Handelt es sich bei folgendem Inhalt um Verkehrs- oder Mobilitätsdaten? Antworte nur mit T (True) oder F (False).\n\n" + "\n\n".join(chunk_lines)

        max_retries = 3
        for attempt in range(max_retries):
            try:
                response = client.models.generate_content_stream(
                    model="gemini-2.0-flash-lite-001",
                    contents=[prompt],
                    config=types.GenerateContentConfig(
                        max_output_tokens=chunk_size,
                        temperature=0
                    )
                )
                result_text = "".join(chunk.text for chunk in response)
                break
            except Exception as e:
                error_str = str(e)
                if "RESOURCE_EXHAUSTED" in error_str or "429" in error_str:
                    print(f"Rate limit reached. Waiting 60 seconds... (Attempt {attempt+1} of {max_retries})")
                    time.sleep(60)
                else:
                    print(f"Error: {error_str}")
                    break
        else:
            df.loc[batch_indices, 'mobilitydata_generated'] = "ERROR"
            continue

        # Process model output
        predictions = list(result_text.strip())

        if all(p in ['T', 'F'] for p in predictions) and len(predictions) == len(batch_indices):
            df.loc[batch_indices, 'mobilitydata_generated'] = predictions
        else:
            df.loc[batch_indices, 'mobilitydata_generated'] = "ERROR"

        key_request_counter += 1
        time.sleep(0.8)

# Main logic
df['mobilitydata_generated'] = None

# First run with chunk_size = 10
all_indices = df.index.tolist()
process_chunks(df, all_indices, chunk_size=10)

# Retry failed ones with chunk_size = 5
retry_indices = df[df['mobilitydata_generated'] == "ERROR"].index.tolist()
process_chunks(df, retry_indices, chunk_size=5)

# Final attempt with chunk_size = 1
retry_indices = df[df['mobilitydata_generated'] == "ERROR"].index.tolist()
process_chunks(df, retry_indices, chunk_size=1)

In [ ]:
# Configuration
requests_per_key = 15
key_count = len(API_KEYS)
current_key_index = 0
key_request_counter = 0
cycle_start_time = time.time()

def build_prompt(prompt_id, chunk_lines):
    if prompt_id == 1:
        return (
            "Handelt es sich bei folgendem Inhalt um Verkehrs- oder Mobilitätsdaten?\n\n" +
            "\n\n".join(chunk_lines) + "\n\n" +
            "Antworte nur mit T für True, F für False oder U für Uncertain und reihe alle Antworten direkt aneinander. "
            "Ohne Leerzeichen, Umbrüche, Texte, Sonderzeichen."
        )
    elif prompt_id == 2:
        return (
            "Handelt es sich bei folgendem Inhalt um Verkehrs- oder Mobilitätsdaten?\n\n" +
            "\n\n".join(chunk_lines) + "\n\n" +
            "Antworte nur mit T für True, F für False oder U für Uncertain und reihe alle Antworten direkt aneinander ohne Leerzeichen, Umbrüche, Texte oder Sonderzeichen zu verwenden. Beispielhaftes Antwortschema:\n\n" +
            "TFTFFTFUTT"
        )
    elif prompt_id == 3:
        return (
            "Untenstehend finden Sie jeweils einen Titel und eine Beschreibung eines Datensatzes."
            "Bewerten Sie für jede Kombination, ob es sich um Verkehrs- oder Mobilitätsdaten handelt.\n\n"
            "Antworten Sie für jede Zeile **nur** mit:\n"
            "- **T** für True = es handelt sich um Mobilitätsdaten\n"
            "- **F** für False = es handelt sich **nicht** um Mobilitätsdaten\n"
            "- **U** für Uncertain = Sie sind sich unsicher in der Zuweisung\n\n"
            "Reihen Sie alle Antworten ohne Leerzeichen, Umbrüche oder andere Zeichen direkt hintereinander. "
            "Geben Sie ausschließlich eine Zeichenkette bestehend aus T, F und U zurück.\n\n"
            "Beispiel für 10 Einträge:\n"
            "TFTFTFTUFT\n\n"
            "Datensätze:\n\n" +
            "\n\n".join(chunk_lines)
        )
    elif prompt_id == 4:
        return (
            "Als Experte für Datenannotation im Bereich Mobilitäts- und Verkehrsdaten ist es Ihre Aufgabe, öffentliche Datensätze daraufhin zu prüfen, ob sie Informationen enthalten, die eindeutig Mobilitäts- oder Verkehrsdaten betreffen. Ihre Einschätzung hilft bei der sachgerechten Klassifizierung dieser Inhalte auf einem nationalen Datenportal.\n"
            "Aufgabe:\n" 
            "Beurteilen Sie, ob es sich bei dem folgenden Datensatz um Mobilitäts- oder Verkehrsdaten handelt.\n\n"
            "Antwortformat:\n"  
            "Antworten Sie **nur mit einer der folgenden Optionen**, ohne zusätzliche Zeichen oder Erläuterungen (T für True, F für False, U für Uncertain):\n\n"
            "- T\n"  
            "- F\n"  
            "- U\n\n"
            "Hinweise:\n" 
            "- Verwenden Sie **U: Uncertain**, wenn die Informationen im Titel oder in der Beschreibung unklar oder nicht ausreichend sind.\n"  
            "- Berücksichtigen Sie Aspekte wie Verkehrsmittel, Infrastruktur, Mobilitätsverhalten oder Verkehrsfluss.\n"  
            "- Geben Sie keine zusätzlichen Erläuterungen – nur die ausgewählte Option.\n\n"
            "Datensatzbeschreibung:\n" +
            "\n\n".join(chunk_lines) +
            "Antwort:"
        )
    elif prompt_id == 5:
        return (
            "Handelt es sich bei folgendem Inhalt um Verkehrs- oder Mobilitätsdaten? "
            "Antworte nur mit T (True), F (False) oder U (Uncertain).\n\n" + "\n\n".join(chunk_lines)
        )
    elif prompt_id == 6:
        return (
            "Handelt es sich bei folgendem Inhalt um Verkehrs- oder Mobilitätsdaten? "
            "Antworte nur mit T (True) oder F (False).\n\n" + "\n\n".join(chunk_lines)
        )
    else:
        raise ValueError("Ungültige Prompt-ID")


def process_chunks(df, indices, chunk_size, prompt_id):
    global current_key_index, key_request_counter, cycle_start_time

    results = []

    for i in tqdm(range(0, len(indices), chunk_size)):
        # Rotate key if needed
        if key_request_counter >= requests_per_key:
            current_key_index += 1
            key_request_counter = 0

            if current_key_index >= key_count:
                elapsed = time.time() - cycle_start_time
                if elapsed < 60:
                    wait_time = int(60 - elapsed)
                    print(f"Max requests per minute reached. Waiting {wait_time} seconds...")
                    time.sleep(wait_time + 1)
                current_key_index = 0
                cycle_start_time = time.time()

        CURRENT_API_KEY = API_KEYS[current_key_index]
        client = genai.Client(api_key=CURRENT_API_KEY)

        batch_indices = indices[i:i + chunk_size]
        chunk_df = df.loc[batch_indices][['dataset_title_DE', 'dataset_description_DE']]

        chunk_lines = chunk_df.apply(
            lambda row: f"Titel: {row['dataset_title_DE']}\nBeschreibung: {row['dataset_description_DE']}",
            axis=1
        ).tolist()

        prompt = build_prompt(prompt_id, chunk_lines)

        max_retries = 3
        for attempt in range(max_retries):
            try:
                response = client.models.generate_content_stream(
                    model="gemini-2.0-flash-lite-001",
                    contents=[prompt],
                    config=types.GenerateContentConfig(
                        max_output_tokens=chunk_size,
                        temperature=0
                        top_p=0.95, # Defaultvalue
                        top_k=64, # Defaultvalue
                        candidate_count=1, # Defaultvalue
                    )
                )
                result_text = "".join(chunk.text for chunk in response)
                break
            except Exception as e:
                error_str = str(e)
                if "RESOURCE_EXHAUSTED" in error_str or "429" in error_str:
                    print(f"Rate limit reached. Waiting 60 seconds... (Attempt {attempt+1} of {max_retries})")
                    time.sleep(60)
                else:
                    print(f"Error: {error_str}")
                    break
        else:
            # All retries failed
            for idx in batch_indices:
                results.append({
                    "index": idx,
                    "mobilitydata_generated": "ERROR",
                    "prompt_id": prompt_id,
                    "chunk_size": chunk_size
                })
            continue

        # Split result
        predictions = list(result_text.strip())
        for rel_idx, prediction in zip(batch_indices, predictions):
            results.append({
                "index": rel_idx,
                "mobilitydata_generated": prediction if prediction in ["T", "F", "U"] else "ERROR",
                "prompt_id": prompt_id,
                "chunk_size": chunk_size
            })

        key_request_counter += 1
        time.sleep(0.8)

    return results


# ----- Main Evaluation -----

# Baseline DataFrame vorbereiten
df['mobilitydata_generated'] = None
all_indices = df.index.tolist()

all_results = []

# Lege neue Spalten für alle Prompt-/Chunk-Kombinationen an
for prompt_id in range(1, 7):
    for chunk_size in [10, 5, 1]:
        col_name = f"mobilitydata_generated_p{prompt_id}_c{chunk_size}"
        df[col_name] = None

# Führe Modellabfragen für alle Kombinationen aus und sammle Ergebnisse
for prompt_id in range(1, 7):
    for chunk_size in [10, 5, 1]:
        print(f"Running Prompt {prompt_id} with chunk_size {chunk_size}")
        result_rows = process_chunks(df.copy(), all_indices, chunk_size, prompt_id)
        all_results.extend(result_rows)

# Ergebnisse in die jeweiligen Spalten schreiben
for row in all_results:
    idx = row['index']
    prompt_id = row['prompt_id']
    chunk_size = row['chunk_size']
    value = row['mobilitydata_generated']
    col_name = f"mobilitydata_generated_p{prompt_id}_c{chunk_size}"
    df.at[idx, col_name] = value

Running Prompt 1 with chunk_size 10


100%|██████████| 15/15 [00:19<00:00,  1.31s/it]


Running Prompt 1 with chunk_size 5


100%|██████████| 30/30 [00:38<00:00,  1.28s/it]


Running Prompt 1 with chunk_size 1


...

100%|██████████| 15/15 [00:19<00:00,  1.30s/it]


Running Prompt 6 with chunk_size 5


100%|██████████| 30/30 [00:38<00:00,  1.27s/it]


Running Prompt 6 with chunk_size 1


100%|██████████| 150/150 [02:56<00:00,  1.18s/it]


In [48]:
analysis = []

# Ergebnis-Spalten automatisch erkennen
result_columns = [col for col in df.columns if col.startswith("mobilitydata_generated_p")]

for col in sorted(result_columns):
    try:
        parts = col.split("_")
        prompt_id = int(parts[2][1:])   # korrekt: 'p1'
        chunk_size = int(parts[3][1:])  # korrekt: 'c10'

        valid = df[df[col].isin(['T', 'F'])].copy()
        valid['prediction'] = valid[col].map({'T': True, 'F': False})

        tp = ((valid['mobilitydata_labelled'] == True) & (valid['prediction'] == True)).sum()
        tn = ((valid['mobilitydata_labelled'] == False) & (valid['prediction'] == False)).sum()
        fp = ((valid['mobilitydata_labelled'] == False) & (valid['prediction'] == True)).sum()
        fn = ((valid['mobilitydata_labelled'] == True) & (valid['prediction'] == False)).sum()
        total = len(valid)

        accuracy = (tp + tn) / total if total > 0 else 0
        precision = tp / (tp + fp) if (tp + fp) > 0 else 0
        recall = tp / (tp + fn) if (tp + fn) > 0 else 0
        f1_score = 2 * (precision * recall) / (precision + recall) if (precision + recall) > 0 else 0

        analysis.append({
            'prompt_id': prompt_id,
            'chunk_size': chunk_size,
            'TP': tp,
            'TN': tn,
            'FP': fp,
            'FN': fn,
            'Total': total,
            'Accuracy': round(accuracy, 4),
            'Precision': round(precision, 4),
            'Recall': round(recall, 4),
            'F1-Score': round(f1_score, 4)
        })
    except Exception as e:
        print(f"Überspringe Spalte {col} wegen Fehler: {e}")

analysis_df = pd.DataFrame(analysis)
analysis_df = analysis_df.sort_values(['prompt_id', 'chunk_size'])

print("\nAuswertung der Prompt-/Chunk-Kombinationen:")
print(analysis_df.to_string(index=False))



Auswertung der Prompt-/Chunk-Kombinationen:
 prompt_id  chunk_size  TP  TN  FP  FN  Total  Accuracy  Precision  Recall  F1-Score
         1           1  25 118   3   4    150    0.9533     0.8929  0.8621    0.8772
         1           5  25  72  47   3    147    0.6599     0.3472  0.8929    0.5000
         1          10  18  57  47   7    129    0.5814     0.2769  0.7200    0.4000
         2           1  25 115   6   4    150    0.9333     0.8065  0.8621    0.8333
         2           5  25  82  38   4    149    0.7181     0.3968  0.8621    0.5435
         2          10  21  65  42   7    135    0.6370     0.3333  0.7500    0.4615
         3           1  24 116   5   5    150    0.9333     0.8276  0.8276    0.8276
         3           5  21  80  37   8    146    0.6918     0.3621  0.7241    0.4828
         3          10  19  76  40  10    145    0.6552     0.3220  0.6552    0.4318
         4           1  19 119   2  10    150    0.9200     0.9048  0.6552    0.7600
         4          

In [43]:
# Write dataframe in new csv-File
df.to_csv(outputdata_file, index=False)

print(f'The file has been successfully saved as {outputdata_file}.')

The file has been successfully saved as data/02_predicted_data.csv.
